# Economic Data per County in NC 

In [ ]:
# zillow data dd properties 
# rural areas from 1 to 3 hours from nearest big city 
# FRED economic data 

In [18]:
# configs and functions

def df_to_csv(df, filename):
    """
    convert df to csv file
    """
    return df.to_csv(filename, index = False)

def get_fred_series_data(api_key, series):
    
    # url
    url = "http://api.stlouisfed.org/geofred/series/data?series_id={0}&api_key={1}&file_type=json".format(series, api_key)
    response = requests.request("GET", url)
    return response

def transform_series_response(response):
  latest_date = list(response.json()['meta']['data'].keys())[0]
  return pd.DataFrame(response.json()['meta']['data'][latest_date])


def get_fred_data(param_list, start_date, end_date):
  df = pdr.DataReader(param_list, 'fred', start_date, end_date)
  return df.reset_index()

fred_api_key = "7b735d0faad802abfee93ec3a7b5188e"

In [19]:
# imports
import requests
from datetime import datetime
import pandas as pd
import pandas_datareader as pdr

# Authorized Building Permits from 2014 to 2019

## Grabbing each counties count of authorized building permits from 2014 to 2015

In [59]:
# pulling data for bubcombe county 
series = 'BPPRIV037021'
get_fred_data(param_list=['BPPRIV037021'], start_date = '2014-01-01', end_date = '2019-12-31')

,DATE,BPPRIV037021
0,2014-01-01,1312.0
1,2015-01-01,1353.0
2,2016-01-01,2422.0
3,2017-01-01,1900.0
4,2018-01-01,1667.0
5,2019-01-01,2096.0


### grabbing series_id for each county

In [67]:
# get series of ids for each county in NC 
response = get_fred_series_data(fred_api_key, series) 

# transform response into a dataframe 
df_of_all_series_ids = transform_series_response(response)

# get all the rows with NC in it 

regex = 'NC'

building_permits_per_county_nc = df_all_series_ids[df_all_series_ids.region.str.contains(regex)].reset_index()

building_permits_per_county_nc

# len(building_permits_per_county_nc['region'].unique().tolist())


,index,region,code,value,series_id
0,1685,"Chatham County, NC",37037,1124,BPPRIV037037
1,1686,"Camden County, NC",37029,75,BPPRIV037029
2,1687,"Cabarrus County, NC",37025,2152,BPPRIV037025
3,1688,"Buncombe County, NC",37021,2793,BPPRIV037021
4,1689,"Brunswick County, NC",37019,5249,BPPRIV037019
...,...,...,...,...,...
95,1795,"Transylvania County, NC",37175,195,BPPRIV037175
96,1796,"Surry County, NC",37171,130,BPPRIV037171
97,1797,"Stokes County, NC",37169,180,BPPRIV037169
98,1798,"Scotland County, NC",37165,52,BPPRIV037165


### grabbing the authorized building permits for each county from 2014 to 2019

In [69]:
# getting data for each county using series_id from building_permits_per_county_nc
series_list = building_permits_per_county_nc['series_id'].tolist()

df_permits_all_counties = get_fred_data(param_list = series_list, 
                          start_date = '2014-01-01', 
                          end_date = '2019-12-31')

df_permits_all_counties

,DATE,BPPRIV037037,BPPRIV037029,BPPRIV037025,BPPRIV037021,BPPRIV037019,BPPRIV037015,BPPRIV037017,BPPRIV037013,BPPRIV037011,...,BPPRIV037185,BPPRIV037183,BPPRIV037181,BPPRIV037179,BPPRIV037177,BPPRIV037175,BPPRIV037171,BPPRIV037169,BPPRIV037165,BPPRIV037159
0,2014-01-01,587.0,20.0,1554.0,1312.0,1973.0,20.0,80.0,170.0,44.0,...,30.0,10125.0,26.0,1425.0,1.0,65.0,73.0,135.0,32.0,288.0
1,2015-01-01,596.0,20.0,1763.0,1353.0,2325.0,18.0,42.0,148.0,47.0,...,48.0,10089.0,74.0,1396.0,1.0,82.0,154.0,75.0,29.0,294.0
2,2016-01-01,707.0,36.0,1918.0,2422.0,2428.0,25.0,41.0,216.0,90.0,...,40.0,11050.0,35.0,1572.0,1.0,104.0,154.0,85.0,9.0,320.0
3,2017-01-01,748.0,52.0,1657.0,1900.0,3342.0,12.0,41.0,179.0,92.0,...,41.0,11335.0,60.0,1537.0,3.0,112.0,113.0,69.0,15.0,403.0
4,2018-01-01,710.0,81.0,2021.0,1667.0,2926.0,11.0,31.0,140.0,118.0,...,55.0,12664.0,44.0,1634.0,3.0,128.0,159.0,69.0,20.0,541.0
5,2019-01-01,902.0,83.0,2714.0,2096.0,3352.0,19.0,52.0,162.0,131.0,...,57.0,9751.0,42.0,1623.0,4.0,118.0,98.0,97.0,24.0,464.0


### pivoting the original dataframe

In [76]:
df_permits_all_counties_melt = pd.melt(df_permits_all_counties, 
                                      id_vars=['DATE'], 
                                      value_vars=series_list, 
                                      var_name='STATE', 
                                      value_name='PERMITS')

df_permits_all_counties_melt

,DATE,STATE,PERMITS
0,2014-01-01,BPPRIV037037,587.0
1,2015-01-01,BPPRIV037037,596.0
2,2016-01-01,BPPRIV037037,707.0
3,2017-01-01,BPPRIV037037,748.0
4,2018-01-01,BPPRIV037037,710.0
...,...,...,...
595,2015-01-01,BPPRIV037159,294.0
596,2016-01-01,BPPRIV037159,320.0
597,2017-01-01,BPPRIV037159,403.0
598,2018-01-01,BPPRIV037159,541.0


### mapping series ids with their respective states

In [82]:
def clean_counties(string):
    if 'County' in string:
        return string[:string.find(' Co')]

def map_id_to_county(string):
    
    reference_table = dict(zip(building_permits_per_county_nc.series_id,
                               building_permits_per_county_nc.region.apply(clean_counties)))
    
    if string in reference_table:
        return reference_table[string]
    
df_permits_all_counties_melt.loc[:, 'STATE'] = df_permits_all_counties_melt['STATE'].apply(map_id_to_county)

In [83]:
df_permits_all_counties_melt

,DATE,STATE,PERMITS
0,2014-01-01,Chatham,587.0
1,2015-01-01,Chatham,596.0
2,2016-01-01,Chatham,707.0
3,2017-01-01,Chatham,748.0
4,2018-01-01,Chatham,710.0
...,...,...,...
595,2015-01-01,Rowan,294.0
596,2016-01-01,Rowan,320.0
597,2017-01-01,Rowan,403.0
598,2018-01-01,Rowan,541.0


In [86]:
df_to_csv(df_permits_all_counties_melt, "authorized-building-permits-2014-2019.csv")

# resources
* https://www.youtube.com/watch?v=nfjzv0cTXH0&ab_channel=AnalyticsAriel
* https://github.com/analyticsariel/projects/blob/master/economic_data/fred_economic_us_data_real_estate.ipynb  
* https://fred.stlouisfed.org/searchresults?st=building%20permit&t=nc%3Bcounty&ob=sr&od=desc